In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import thinkstats2
import thinkplot
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, confusion_matrix
from sklearn.model_selection import train_test_split

# Assignment 4 - Classification

For this assignment, you'll need to perform a classification on a dataset, as well as do some prep work on the data. 

The exact steps of what you need to do are flexible and up to you to some degree, however you should consider some of the important things we've mentioned recently, such as:
<ul>
<li> Is the target balanced?
<li> Are there missing or erroneous values?
<li> Are there categorical or numerical features?
<li> Is there colinearity?
<li> Are there outliers?
<li> Should we normalize? 
<li> Do the distributions of the features give any indication that some may need work? 
</ul>

Basically, the data is in the original, potentially dirty, format, and you need to find what should be cleaned, and do the cleaning. There is not one "right" answer to what has to be done, and you'll probably need to do it with at least something of an iterative process - find an issue, correct it, check the data, repeat.

<b>The target that we are predicting is the loan_status. </b>

## Grading

The grades will be broken down as follows:
<ul>
<li> <b>30%</b> - a working predictive model with a reasonable level of accuracy. 
    <ul>
    <li> For the accuracy part, it will not be explicitly graded, but compared to all the others. If you're in the same general range, that's good - if yours is drastically less accurate (or, I guess more accurate), then I'll adjust. There won't be a comparison of "this person is 72.3% and this person is only 71.8% accurate, they fail."
    <li> This data is larger than most of the small sample sets, so random variations due to train-test splits shouldn't be too bad. (If you're a keener you could loop)
    <li> I will use the F1 score as the accuracy metric. 
    </ul>
<li> <b>40%</b> - a clear and readable description of what steps you took to prepare the data, and a brief not on the rationale behind it. Did you do a log transformation to a skewed feature, did you remove outliers, did you remove a feature that has a lot of missing values? Please put this somewhere obvious and readable, consider the goal of the assignment to explain your process to me. 
    <ul>
    <li> E.g. "The target data was imbalanced, so I tried several resampling methods and chose the one with the highest accuracy of the resulting model", "the feature X had the same value for 95% of records, so I dropped it". 
    <li> In this, please also state if you see a group that appears to be a good credit risk, and a group that is a bad credit risk, and indicate the evidence showing that. Please do this other than the most simplistic way - more money = better credit. If there are no such groups, state why you think this. This will likely be about 3 - 5 statements or points, you should provide evidence from the data, but it does not need to be an essay. (You may want to consider this question after you're pretty much done with the data prep and modelling.)
    </ul>
<li> <b>30%</b> - allowing your model to be "deployed". At the bottom of this file there is a small block of code to load in some test data (that I have), and calculate your accuracy. Your contribution to this part is to have a model that is ready to make predictions. Some specifics to consider:
    <ul>
    <li> The test data will be in exactly the same format as the dataset you're given. So any steps that you took to prepare your data for modelling will need to be mirroed here, so the new  data can be predicted. The easiest way to do this is to use a pipeline, but it is up to you. Remember the model only accepts data that is in a certain format - the one that you had the data in when it was trained, so when making predictions you need to make sure that is true. 
    <li> Since I'm providing test data, and your task is to just create a model, think about how that might impact your train-test splitting, both as you're developing and for the final product. 
    <li> Once the model is trained it should predict any data that is in the right format, so I should be able to provide any proper dataset, click run on that testing cell, and get predictions. I will not be doing anything that will purposefully make this harder or trickier, like including data that has errors or is in the wrong format, I just split the entire set of data, reserved part of it to test accuracy, and gave the rest to you. Things that you've changed (e.g. dropping a column, one-hot encoding) do need to be replicated at some point before the model can accept the new data to predict. In general we <b>do</b> want to check in our processing that our input is valid, this is kind of a junior introduction to that. 
    </ul>
</ul

###My Ideas
1.clean the data(drop the columns which are not usefull, and filtering the outliers)


## Load Credit Data

In [14]:
df = pd.read_csv("train.csv")
df.head()

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,26967,27,120000,MORTGAGE,3.0,PERSONAL,B,14000,11.99,0,0.12,N,10
1,8449,24,58000,MORTGAGE,8.0,DEBTCONSOLIDATION,A,15700,7.90,0,0.27,N,3
2,20983,30,45000,RENT,11.0,MEDICAL,C,5000,12.73,1,0.11,Y,8
3,19612,35,35000,MORTGAGE,5.0,EDUCATION,B,5600,11.49,0,0.16,N,6
4,30371,38,55000,RENT,0.0,EDUCATION,C,6500,11.03,0,0.12,Y,12


In [15]:
df.describe()

,Unnamed: 0,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,26064.000000,26064.000000,2.606400e+04,25350.000000,26064.000000,23575.000000,26064.000000,26064.000000,26064.000000
mean,16307.473258,27.738375,6.602795e+04,4.791440,9595.886088,11.006839,0.216812,0.170264,5.805210
std,9396.129345,6.381305,6.265098e+04,4.100277,6321.894580,3.245217,0.412082,0.106673,4.055158
min,0.000000,20.000000,4.080000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,8202.250000,23.000000,3.879375e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,16324.500000,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,24458.250000,30.000000,7.920000e+04,7.000000,12200.000000,13.470000,0.000000,0.230000,8.000000
max,32580.000000,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [16]:
df=df.drop(columns={'Unnamed: 0','cb_person_cred_hist_length'})
df=df[df["loan_percent_income"] < 0.65]
df=df[df["person_emp_length"] < 50]
df=df[df["person_age"] < 65]
#remove null values
df.dropna(axis=0, inplace=True)
df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income
count,22866.000000,2.286600e+04,22866.000000,22866.000000,22866.000000,22866.000000,22866.000000
mean,27.670428,6.645707e+04,4.792049,9660.015963,11.035021,0.215254,0.169102
std,6.008327,5.065783e+04,4.030043,6318.730899,3.234770,0.411007,0.105090
min,20.000000,4.080000e+03,0.000000,500.000000,5.420000,0.000000,0.000000
25%,23.000000,3.960000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000
50%,26.000000,5.600000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000
75%,30.000000,8.000000e+04,7.000000,12500.000000,13.480000,0.000000,0.230000
max,64.000000,1.782000e+06,38.000000,35000.000000,23.220000,1.000000,0.640000


In [17]:
df['person_home_ownership'] = df['person_home_ownership'].astype('category')
df['loan_intent'] = df['loan_intent'].astype('category')
df['loan_grade'] = df['loan_grade'].astype('category')

In [18]:
y = np.array(df["loan_status"]).reshape(-1,1)
x = np.array(df.drop(columns="loan_status"))
y.shape,x.shape

((22866, 1), (22866, 10))

In [19]:
sns.pairplot(df, hue="loan_status")

In [20]:
df = pd.get_dummies(df, drop_first=True)
df.sample(10)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
22194,34,70000,1.0,6000,11.89,0,0.09,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1284,22,30000,3.0,10000,13.49,1,0.33,0,0,0,...,0,0,0,0,1,0,0,0,0,0
19856,23,60000,1.0,1600,13.99,0,0.03,0,0,1,...,1,0,0,0,1,0,0,0,0,1
977,23,50000,5.0,10000,8.94,0,0.20,0,0,1,...,1,0,0,0,0,0,0,0,0,0
10208,25,25000,5.0,9000,11.49,1,0.36,0,0,1,...,1,0,0,1,0,0,0,0,0,0
11792,24,29120,0.0,5000,10.99,0,0.17,0,0,1,...,0,0,1,1,0,0,0,0,0,0
7950,29,100000,1.0,10000,14.79,0,0.10,0,0,0,...,0,1,0,0,1,0,0,0,0,0
7488,44,28000,0.0,3500,8.94,0,0.13,0,1,0,...,0,0,1,0,0,0,0,0,0,0
10614,23,62500,7.0,26000,11.71,1,0.42,0,0,1,...,1,0,0,1,0,0,0,0,0,0
19798,24,64000,7.0,3000,15.21,1,0.05,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [21]:
y = np.array(df["loan_status"]).reshape(-1,1)
x = np.array(df.drop(columns={"loan_status"}))
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.33)

In [22]:
def convert_prob_to_label(prob, cutoff = 0.5):
    label = []
    for i in range(len(prob)):
        if prob[i] > cutoff:
            label.append(1)
        else:
            label.append(0)
    return label

In [23]:
from sklearn.metrics import log_loss, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import binarize
from sklearn.pipeline import make_pipeline


In [25]:
m2 = make_pipeline( MinMaxScaler(),
                    LogisticRegression()
                    )

# Use pipeline like it is a model. 
m2.fit(xTrain, yTrain.ravel())
m2_pred = m2.predict(xTest)

m2_label = convert_prob_to_label(np.array(m2_pred))
m2_matrix = confusion_matrix(yTest,m2_label)
sns.heatmap(m2_matrix, annot=True)

print("F1:",f1_score(yTest, labels))
print("Acc:",accuracy_score(yTest, labels))
print("Recall:",recall_score(yTest, labels))
print("Precision:",precision_score(yTest, labels))

NameError: name 'labels' is not defined

### Accuracy Test

Replace the green part with whatever you need to transform the fresh data into the format needed for your model to predict. 

<b>Note:</b> you could test and make sure that things are working OK here by taking part of your dataset, saving it in a separate CSV, and running it through here as a piece of test data. The accuracy results would be useless, but you'd confirm that the model works properly. 

In [ ]:
d_test = pd.read_csv("test.csv")

######################
# Insert whatever you need to do to prep the data here. 
# It might be nothing if you have a big pipeline. 
# Ensure that at the end your data is in two arrays - xtest and ytest
# Ensure the name of your final model is "model"
# Each should contain the ENTIRE x or y dataset in the test data. 
#
# This part should be reusable - any dataset in this format should just run
# and generate predictions and accuracy stats. 
# Please DO NOT do any train-test splitting here. 
#######################

# This should work once you're done, as is. 
# uncomment the stuff below when ready to run.
#print("F1:",f1_score(ytest, prediction_labels))
#conf_matrix = confusion_matrix(ytest, prediction_labels)
#sns.heatmap(conf_matrix, annot=True)